In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Output, Input, State
import plotly.graph_objs as go
import pandas as pd

from pathlib import Path
import os

def find_latest_csv(base_dir):
    base_path = Path(base_dir)
    csv_files = base_path.rglob('*.csv')
    try:
        latest_file = max(csv_files, key=os.path.getmtime)
        return str(latest_file)
    except ValueError:
        return None


# GLOBAL VARIABLES
df = pd.DataFrame()
last_csv_file = None
last_file_size = 0
last_row_count = 0
last_db_save_time = 0
current_folder = ""

# Constants
chosen_folder = "./output/20250524"
app = dash.Dash(__name__)

app.layout = html.Div([

    dcc.Graph(id='graph'),

    dcc.Interval(
        id='interval-component',
        interval=1*1000,  # 1 x 1000 ms
        n_intervals=0
    ),

    # # Store to hold the folder path
    # dcc.Store(id="folder-path", data="./output/20250524")
])

@app.callback(
    Output('graph', 'figure'),
    Input('interval-component', 'n_intervals'),
)
def update_graph(n):
    global df, last_db_save_time, last_csv_file, global_start_time, last_file_size, last_row_count
    
    force_columns = {
        'compr1data': 'compression_force_1',
        'compr2data': 'compression_force_2', 
        'ejectdata': 'ejection_force',
        'precompdata': 'pre_compression_force',
        'COMPR1DATA': 'compression_force_1',
        'COMPR2DATA': 'compression_force_2',
        'EJECTDATA': 'ejection_force',
        'PRECOMPDATA': 'pre_compression_force',
        'compression1': 'compression_force_1',
        'compression2': 'compression_force_2',
        'ejection': 'ejection_force',
        'precompression': 'pre_compression_force',
    }

    latest_csv = find_latest_csv(chosen_folder)

    ### DEAL WITH UPDATING LATER ###
    # if not latest_csv:
    #     print("No CSV files found in the specified folder.")
    #     return go.Figure()

    # latest_csv_size = Path(latest_csv).stat().st_size

    # file_changed = (latest_csv != last_csv_file) or (latest_csv_size != last_file_size)

    # if file_changed:
    #     new_data = pd.read_csv(latest_csv)
    #     new_data.columns = new_data.columns.str.lower().str.strip()
    ################################

    new_data = pd.read_csv(latest_csv)
    new_data.columns = new_data.columns.str.lower().str.strip()

    # pre_compression_graph = generate_punch_stats_figure("pre_compression", df)

    df = new_data




    return go.Figure()

    # try:
        
    #     data = ticker.history(period="1d", interval="1m").tail(20)
    #     data['SMA_5'] = data['Close'].rolling(window=5).mean()
    #     data['SMA_10'] = data['Close'].rolling(window=10).mean()

    #     fig = go.Figure()

    #     # Add the figures
    #     fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines+markers', name='USD/JPY'))

    #     # Change y axis
    #     fig.update_yaxes(title_text='Rate', range=[data['Close'].min() - 0.1, data['Close'].max() + 0.1])

    #     # Label all x ticks
    #     fig.update_xaxes(tickvals=data.index, ticktext=[t.strftime('%H:%M') for t in data.index], tickangle=315)
        
    #     fig.update_layout(title='USD/JPY (Live) with SMA', xaxis_title='Time', yaxis_title='Rate')

    #     return fig

    # except Exception as e:
    #     print("Error:", e)
    #     return go.Figure()

if __name__ == '__main__':
    app.run(debug=True)

In [2]:
import pandas as pd
from pathlib import Path
import os

def find_latest_csv(base_dir):
    base_path = Path(base_dir)
    csv_files = base_path.rglob('*.csv')
    try:
        latest_file = max(csv_files, key=os.path.getmtime)
        return str(latest_file)
    except ValueError:
        return None

df = pd.read_csv(find_latest_csv('./output/20250515'))

In [8]:
new_data = df.copy()
new_data['timestamp'] = pd.to_numeric(new_data['timestamp'], errors='coerce')
new_data['relative_time'] = (new_data['timestamp']) / 1000  # Convert ms to seconds
new_data['absolute_time'] = new_data['timestamp']

new_data

,timestamp,actualtime(ms),compr1data,compr2data,ejectdata,precompdata,compr1punchno,compr2punchno,ejectpunchno,precomppunchno,...,relative_time,absolute_time,datetime,time_str,compression_force_1,compression_force_2,ejection_force,pre_compression_force,sticking,normal
0,19170000,2025-05-15T15:11:59.170,0,0,776,0,0,0,8,0,...,19170.0,19170000,2025-05-15 15:11:59.170,15:11:59.170000,0,0,776,0,0,1
1,19172000,2025-05-15T15:11:59.172,0,0,639,0,0,0,8,0,...,19172.0,19172000,2025-05-15 15:11:59.172,15:11:59.172000,0,0,639,0,0,1
2,19174000,2025-05-15T15:11:59.174,0,0,894,0,0,0,8,0,...,19174.0,19174000,2025-05-15 15:11:59.174,15:11:59.174000,0,0,894,0,0,1
3,19176000,2025-05-15T15:11:59.176,0,0,902,0,0,0,8,0,...,19176.0,19176000,2025-05-15 15:11:59.176,15:11:59.176000,0,0,902,0,0,1
4,19178000,2025-05-15T15:11:59.178,0,0,819,0,0,0,8,0,...,19178.0,19178000,2025-05-15 15:11:59.178,15:11:59.178000,0,0,819,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,19462000,2025-05-15T15:11:59.462,0,0,792,0,0,0,1,0,...,19462.0,19462000,2025-05-15 15:11:59.462,15:11:59.462000,0,0,792,0,0,1
1109,19464000,2025-05-15T15:11:59.464,0,0,901,0,0,0,1,0,...,19464.0,19464000,2025-05-15 15:11:59.464,15:11:59.464000,0,0,901,0,0,1
1110,19466000,2025-05-15T15:11:59.466,0,0,877,0,0,0,1,0,...,19466.0,19466000,2025-05-15 15:11:59.466,15:11:59.466000,0,0,877,0,0,1
1111,19468000,2025-05-15T15:11:59.468,0,0,839,0,0,0,1,0,...,19468.0,19468000,2025-05-15 15:11:59.468,15:11:59.468000,0,0,839,0,0,1


In [ ]:
new_data.columns

We're interested in 

- `precompdata`
- `precomppunchno`

In [10]:
# Pivot table
# new_data[['compr1data', 'compr1punchno']].groupby(by=['compr1punchno']).mean()
new_data.pivot_table(columns='compr1punchno', values='compr1data', aggfunc=['mean', 'max']).reset_index()

index mean                                         max  \
compr1punchno                0       1      5        6      7        8   0   
0              compr1data  0.0  103.55  103.7  102.075  104.4  100.425   0   

                                        
compr1punchno    1    5    6    7    8  
0              126  118  126  132  120

In [ ]:

# Group by precompression punch number and calculate mean
xd = new_data[['precompdata', 'precomppunchno']].groupby(by=['precomppunchno']).mean()

# Remove 0 from index
xd = xd[xd.index != 0]
xd.to_json()

'{"precompdata":{"1":227.7333333333,"5":222.7333333333,"6":229.3333333333,"7":226.2,"8":226.4666666667}}'

In [5]:
new_data[['precompdata', 'precomppunchno']].groupby(by=['precomppunchno']).max()

,precompdata
precomppunchno,
0,0
1,249
5,249
6,249
7,247
8,247


In [ ]:
new_data = (
    df.copy()
    .assign(
        timestamp=pd.to_numeric(df['timestamp'], errors='coerce'),
        relative_time=lambda x: x['timestamp'] / 1000,
        absolute_time=lambda x: x['timestamp']
    )
)

new_data